In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, root
from climate import EarthModel, fit_eruption_data
from numpy.random import exponential
matplotlib.rcParams.update(_VSCode_defaultMatplotlib_Params)
matplotlib.rcParams.update({'font.size': 14})

In [ ]:
def seconds(years):
    return years * 365.25 * 24 * 3600

In [ ]:
# Simulate eruptions
np.random.seed(42)
test_repose_times = np.array(
    [100, 50, 20, 20, 50, 100]
) * 365.25 * 24 * 3600
zone_eruption_times = np.array([
    exponential(test_repose_times[i], 200).cumsum() for i in range(6)
])
eruption_zones = []
eruption_times = []
for i in range(6):
    for t in zone_eruption_times[i]:
        eruption_zones.append(i)
        eruption_times.append(t)

In [ ]:
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
model.build_eruptions(eruption_zones, eruption_times)
model.t0 = 0
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'
model.build()
model.solve()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
t_years = np.linspace(0, model.tf, model.tn + 1) / 3.154e+7
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=1.5, label='Zone {}'.format(i + 1), zorder=1)

ax.legend(bbox_to_anchor=(1, 1))
ax.set_xlim((0, model.tf))
# Plot zone 0 eruption times as vertical lines
for i in range(6):
    for t in zone_eruption_times[i]:
        if t < model.tf:
            ax.axvline(
                t / 3.154e+7, color='C{}'.format(i),
                zorder=0, linewidth=1.0, linestyle='dashed', alpha=0.5
            )
ax.set_xlim((0, t_years[-1]))
ax.set_xlabel('Time (years)')
ax.set_ylabel('Temperature (K)')
fig.savefig('../Figures/stochastic_eruptions.pdf', bbox_inches='tight')

In [ ]:
Ts = np.linspace(model.temp_i - 10, model.temp_0 + 10, 100)
alphas = [model.alpha(T) for T in Ts]
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(Ts, [alpha[0] for alpha in alphas], label='Zone 1', linewidth=2)
ax.plot(Ts, [alpha[1] for alpha in alphas], label='Zone 2', linewidth=2)
ax.plot(Ts, [alpha[2] for alpha in alphas], label='Zone 3', linewidth=2)
ax.plot(Ts, [alpha[3] for alpha in alphas], label='Zone 4', linewidth=2)
ax.plot(Ts, [alpha[4] for alpha in alphas], label='Zone 5', linewidth=2)
ax.plot(Ts, [alpha[5] for alpha in alphas], label='Zone 6', linewidth=2)
ax.axvline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axvline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.axvspan(220, 250, alpha=0.2, color='deepskyblue')
ax.axvspan(280, 290, alpha=0.2, color='C1')
ax.legend(bbox_to_anchor=(1, 1))
ax.set_xlim((240, 290))
ax.set_ylim((0, 0.7))
ax.text(245, 0.65, '$\\alpha = \\alpha_i$', ha='center', va='center')
ax.text(265, 0.65, '$\\alpha = \\alpha_0 + (\\alpha_i - \\alpha_0)\\frac{(T - T_0)^2}{(T_i - T_0)^2}$', ha='center',va='center')
ax.text(285, 0.65, '$\\alpha = \\alpha_0$', ha='center', va='center')
ax.set_xlabel('Temperature (K)')
ax.set_ylabel('Albedo')
fig.savefig('../figures/albedo.pdf', bbox_inches='tight')

In [ ]:
# Solve for warm equilibrium
T0 = np.ones(model.size) * 280
F = lambda T: model.flux_balance_albedo(0, T)
warm_sol = root(F, T0)

# Solve for unstable equilibrium
T0 = np.ones(model.size) * 250
F = lambda T: model.flux_balance_albedo(0, T)
unstable_sol = root(F, T0)

# Solve for snowball equilibrium
T0 = np.ones(model.size) * 200
F = lambda T: model.flux_balance_albedo(0, T)
cold_sol = root(F, T0)

fig, ax = plt.subplots(figsize=(8, 6))
for i, x in enumerate(warm_sol.x):
    ax.axhline(x - 273.15, color='red')

for i, x in enumerate(unstable_sol.x):
    ax.axhline(x - 273.15, color='black')

for i, x in enumerate(cold_sol.x):
    ax.axhline(x - 273.15, color='blue')

In [ ]:
# Albedo effects
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
model.temp_i = 250 # K
model.temp_0 = 280 # K
model.t0 = 0
model.tf = 20 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'

# Set up plot
fig = plt.figure(figsize=(8 * 3, 6))
gs = fig.add_gridspec(nrows=1, ncols=3)
t_years = np.linspace(0, model.tf, model.tn + 1) / 3.154e+7

# Warm earth equilibrium
model.T0 = unstable_sol.x + 1
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
ax = fig.add_subplot(gs[0, 0])
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i + 1), zorder=1)
ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.set_xlim((0, t_years[-1]))
ax.set_ylim((230, 290))
ax.set_xlabel('Time (years)')
ax.set_ylabel('Temperature (K)')
ax.set_title('Warm Earth Equilibrium')

# Unstable equilibrium
model.T0 = unstable_sol.x
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
ax = fig.add_subplot(gs[0, 1])
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i + 1), zorder=1)
ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.set_xlim((0, t_years[-1]))
ax.set_ylim((230, 290))
ax.set_xlabel('Time (years)')
ax.set_title('Unstable Equilibrium')

# Snowball earth equilibrium
model.T0 = unstable_sol.x - 1
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
ax = fig.add_subplot(gs[0, 2])
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i + 1), zorder=1)
ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.legend(bbox_to_anchor=(1, 1))
ax.set_xlim((0, t_years[-1]))
ax.set_ylim((230, 290))
ax.set_xlabel('Time (years)')
ax.set_title('Snowball Earth Equilibrium')
fig.savefig('../figures/albedo_equilibria_3plots.pdf', bbox_inches='tight')

In [ ]:
# Albedo effects
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
model.temp_i = 250 # K
model.temp_0 = 280 # K
model.t0 = 0
model.tf = 20 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'

# Set up plot
fig, ax = plt.subplots(figsize=(8, 6))
t_years = np.linspace(0, model.tf, model.tn + 1) / 3.154e+7

# Warm earth equilibrium
model.T0 = unstable_sol.x + 1
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, zorder=1,
        linestyle='dashed',
        color='C{}'.format(i)
    )

# Unstable equilibrium
model.T0 = unstable_sol.x
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i + 1), zorder=1,
        color='C{}'.format(i)
    )

# Snowball earth equilibrium
model.T0 = unstable_sol.x - 1
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=2, zorder=1,
        linestyle='dotted',
        color='C{}'.format(i)
    )

ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.legend(bbox_to_anchor=(1, 1))
ax.set_xlim((0, t_years[-1]))
ax.set_ylim((230, 290))
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Time (years)')
ax.text(19.7, 283, 'Warm Earth Equilibrium', fontsize=12, ha='right')
ax.text(19.7, 260, 'Unstable Equilibrium', fontsize=12, ha='right')
ax.text(19.7, 238, 'Snowball Earth Equilibrium', fontsize=12, ha='right')
ax.axhspan(220, 250, alpha=0.2, color='deepskyblue')
ax.axhspan(280, 290, alpha=0.2, color='C1')
fig.savefig('../figures/albedo_equilibria_1plot.pdf', bbox_inches='tight')

In [ ]:
# Albedo effects plus eruptions
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
model.temp_i = 250 # K
model.temp_0 = 280 # K
model.t0 = 0
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'

fig = plt.figure(figsize=(8 * 2, 6))
gs = fig.add_gridspec(nrows=1, ncols=2)
t_years = np.linspace(0, model.tf, model.tn + 1) / 3.154e+7

# Simulate stable eruptions
np.random.seed(42)
test_repose_times = np.array(
    # [80, 30, 10, 10, 30, 80]
    # [100, 50, 20, 20, 50, 100]
    [150, 75, 50, 50, 75, 150]
) * 365.25 * 24 * 3600
zone_eruption_times = np.array([
    exponential(test_repose_times[i], 200).cumsum() for i in range(6)
])
eruption_zones = []
eruption_times = []
for i in range(6):
    for t in zone_eruption_times[i]:
        eruption_zones.append(i)
        eruption_times.append(t)

model.build_eruptions(eruption_zones, eruption_times)
model.build_ode(
    lambda t, T: model.flux_balance_eruptions_albedo(t, T)
)
model.solve()
ax = fig.add_subplot(gs[0, 0])
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=1.5, label='Zone {}'.format(i), zorder=1)
for i in range(6):
    for t in zone_eruption_times[i]:
        if t <= model.tf:
            ax.axvline(
                t / 3.154e+7, color='C{}'.format(i),
                zorder=0, linewidth=1.0, linestyle='dashed', alpha=0.5
            )
ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.axhspan(220, 250, alpha=0.2, color='deepskyblue')
ax.axhspan(280, 290, alpha=0.2, color='C1')
ax.set_ylim((220, 290))
ax.set_xlim((0, t_years[-1]))
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Time (years)')
ax.set_title('Stable Eruption Frequency')

# Simulate semi-stable eruptions
np.random.seed(42)
test_repose_times = np.array(
    # [80, 30, 10, 10, 30, 80]
    [100, 50, 20, 20, 50, 100]
    # [150, 75, 50, 50, 75, 150]
) * 365.25 * 24 * 3600
zone_eruption_times = np.array([
    exponential(test_repose_times[i], 200).cumsum() for i in range(6)
])
eruption_zones = []
eruption_times = []
for i in range(6):
    for t in zone_eruption_times[i]:
        eruption_zones.append(i)
        eruption_times.append(t)

model.build_eruptions(eruption_zones, eruption_times)
model.build_ode(
    lambda t, T: model.flux_balance_eruptions_albedo(t, T)
)
model.solve()
ax = fig.add_subplot(gs[0, 1])
for i in range(model.size):
    ax.plot(
        t_years, model.T[:, i],
        linewidth=1.5, label='Zone {}'.format(i + 1), zorder=1)
for i in range(6):
    for t in zone_eruption_times[i]:
        if t <= model.tf:
            ax.axvline(
                t / 3.154e+7, color='C{}'.format(i),
                zorder=0, linewidth=1.0, linestyle='dashed', alpha=0.5
            )
ax.axhline(model.temp_0, color='black', linewidth=2, linestyle='dashed')
ax.axhline(model.temp_i, color='black', linewidth=2, linestyle='dashed')
ax.axhspan(220, 250, alpha=0.2, color='deepskyblue')
ax.axhspan(280, 290, alpha=0.2, color='C1')
ax.legend(bbox_to_anchor=(1, 1))
ax.set_xlim((0, t_years[-1]))
ax.set_ylim((220, 290))
ax.set_xlabel('Time (years)')
ax.set_title('Unstable Eruption Frequency')
fig.savefig('../figures/eruptions_albedo.pdf', bbox_inches='tight')